In [6]:
import numpy as np
import sympy as sp

First, let's derive the inverse X matrix that maps from fluxes to "a" coefficients:

In [19]:
X = np.vander([-1, 0, 1], 3)
np.linalg.inv(X)

array([[ 0.5, -1. ,  0.5],
       [-0.5,  0. ,  0.5],
       [ 0. ,  1. ,  0. ]])

Then, let's propagate the uncertainties:

In [20]:
np.linalg.inv(np.dot(X.T, X))

array([[ 1.5,  0. , -1. ],
       [ 0. ,  0.5,  0. ],
       [-1. ,  0. ,  1. ]])

This means that a_1 and a_2 are independent with variances 0.5 and 1.5 respectively.

In [24]:
f_minus, f_max, f_plus = sp.symbols("f_minus f_max f_plus")
f = sp.Matrix([[f_minus], [f_max], [f_plus]])
X_s = sp.Matrix(X)
X_s**-1 * f

Matrix([
[-f_max + f_minus/2 + f_plus/2],
[        -f_minus/2 + f_plus/2],
[                        f_max]])

In [25]:
(X_s.T * X_s)**-1

Matrix([
[3/2,   0, -1],
[  0, 1/2,  0],
[ -1,   0,  1]])